In [44]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import IDF
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [45]:
def read_file(fileUrl, spark):
    df = spark.read.csv(fileUrl, sep=",", inferSchema=True, header=False)
    return df

In [3]:
def pre_process(df):
    df = df.withColumnRenamed('_c0', "id").withColumnRenamed('_c1', 'label').withColumnRenamed('_c2', 'tweet')
    
    df = df.withColumn('tweet', regexp_replace('tweet', '[^a-z0-9A-Z`~!@#$%&<>?., ]', ''))
    df = df.withColumn('tweet', regexp_replace('tweet', '[0-9`~!@#$%&<>?,\']', ''))
    df = df.withColumn('tweet', regexp_replace('tweet', 'http://*.*.com', ''))
    df = df.withColumn('tweet', regexp_replace('tweet', 'www.*.com', ''))
    df = df.withColumn('tweet', regexp_replace('tweet', '\.', ''))
    
    tokenizer = Tokenizer(inputCol="tweet", outputCol="words")
    wordData = tokenizer.transform(df)
    
    remover = StopWordsRemover(inputCol="words", outputCol="word_clean")
    word_clean_data = remover.transform(wordData)
    
    count = CountVectorizer(inputCol="word_clean", outputCol="rawFeatures")
    model = count.fit(word_clean_data)
    
    featurizedData = model.transform(word_clean_data)
    idf = IDF(inputCol="rawFeatures", outputCol="features")
    idfModel = idf.fit(featurizedData)
    rescaledData = idfModel.transform(featurizedData)
    
    return rescaledData

In [4]:
def train_test_split(df):
    seed = 0
    trainDf, testDf = df.randomSplit([0.7, 0.3], seed)
    return trainDf, testDf

In [5]:
def details_table(train_predictions, test_predictions):
    train_predictions.groupBy('label', 'prediction').count().show()
    test_predictions.groupBy('label', 'prediction').count().show()

In [6]:
def evaluate_model(predictions, labelCol="label", predictionCol="prediction"):
    evaluator = BinaryClassificationEvaluator(rawPredictionCol=predictionCol, labelCol=labelCol, metricName="areaUnderROC")
    roc = evaluator.evaluate(predictions)
    
    evaluator = MulticlassClassificationEvaluator(predictionCol=predictionCol, labelCol=labelCol, metricName="f1")
    f1 = evaluator.evaluate(predictions)
    
    evaluator = MulticlassClassificationEvaluator(predictionCol=predictionCol, labelCol=labelCol, metricName="accuracy")
    accuracy = evaluator.evaluate(predictions)
    
    return {"ROC": roc, "F1": f1, "Accuracy": accuracy}

In [7]:
def logistic_regression(train_data, test_data):
    lr = LogisticRegression(maxIter=15)
    paramGrid_lr = ParamGridBuilder().build()
    
    crossval_lr = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid_lr, evaluator=BinaryClassificationEvaluator(), numFolds=8)
    cvModel_lr = crossval_lr.fit(train_data)
    
    best_model_lr = cvModel_lr.bestModel
    train_fit_lr = best_model_lr.transform(train_data)
    train_summary = evaluate_model(train_fit_lr)
    
    predictions_lr = cvModel_lr.transform(test_data)
    test_summary = evaluate_model(predictions_lr)

    details_table(train_fit_lr, predictions_lr)
    
    return train_summary, test_summary

In [54]:
if __name__ == "__main__":
    spark = SparkSession.builder.appName("SocialApp").getOrCreate()
    url = "C:/Users/Causario/Desktop/RedSocial/social_app/data/sentiment_analysis/prueba.txt"  # Provide the path to file
    df = read_file(url, spark)
    df = pre_process(df)
    train_data, test_data = train_test_split(df)
    train_summary, test_summary = logistic_regression(train_data, test_data)
    spark.stop()

DataFrame[_c0: string]


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `tweet` cannot be resolved. Did you mean one of the following? [`id`].;
'Project [id#393, regexp_replace('tweet, [^a-z0-9A-Z`~!@#$%&<>?., ], , 1) AS tweet#398]
+- Project [_c0#391 AS id#393]
   +- Relation [_c0#391] csv
